In [1]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import torch
from scipy.ndimage import gaussian_filter
from tqdm.notebook import tqdm
import plotly
import plotly.figure_factory as ff
import numpy as np
import uuid
import monai

In [2]:
import sys
sys.path.append('../')
from utils.datasets import *
from utils.data_augmentation import *

In [3]:
def compute_covariance(input_data):
    """
    Compute Covariance matrix of the input data
    """
    n = input_data.size(0)  # batch_size

    # Check if using gpu or cpu
    if input_data.is_cuda:
        device = torch.device('cuda')
    else:
        device = torch.device('cpu')

    id_row = torch.ones(n).resize(1, n).to(device=device)
    sum_column = torch.mm(id_row, input_data)
    mean_column = torch.div(sum_column, n)
    term_mul_2 = torch.mm(mean_column.t(), mean_column)
    d_t_d = torch.mm(input_data.t(), input_data)
    c = torch.add(d_t_d, (-1 * term_mul_2)) * 1 / (n - 1)

    return c

## Covariances

In [4]:
%matplotlib inline

In [5]:
vendor = "A"
normalization="none"
data_mod=""
verbose=False

add_depth = False
batch_size = 1

data_augmentation = "none"
img_size, crop_size = 224, 224 # We will take original image not transformed one
mask_reshape_method = "padd"

train_aug, train_aug_img, val_aug = data_augmentation_selector(
    data_augmentation, img_size, crop_size, mask_reshape_method, verbose=verbose
)

dataset = f"mms_vendor{vendor}{data_mod}"

only_end = False if "full" in dataset else True
unlabeled = True if "unlabeled" in dataset else False
c_centre = find_values(dataset, "centre", int)
c_vendor = find_values(dataset, "vendor", str)


train_dataset = MMs3DDataset(
    partition="Training", transform=train_aug, img_transform=train_aug_img, 
    normalization=normalization, add_depth=add_depth, 
    is_labeled=(not unlabeled), centre=c_centre, vendor=c_vendor, 
    end_volumes=only_end, data_relative_path="../"
)

train_loader = DataLoader(
    train_dataset, batch_size=batch_size, pin_memory=True,
    shuffle=False, collate_fn=train_dataset.custom_collate
)

In [6]:
batch = next(iter(train_loader))

In [7]:
slice_indx = 5

batch_image = batch["image"]
print(f"[Image] batch shape: {batch_image.shape}")
first_volume = batch_image[0]
print(f"[Image] first_volume: {first_volume.shape}")
first_slice = first_volume[slice_indx]
print(f"[Image] first_slice: {first_slice.shape}")


print("---------------------------------------------")

batch_mask = batch["label"]
print(f"[Image] batch shape: {batch_mask.shape}")
first_volume_mask = batch_mask[0]
print(f"[Image] first_volume: {first_volume_mask.shape}")
first_slice_mask = first_volume_mask[slice_indx]
print(f"[Image] first_slice: {first_slice_mask.shape}")

[Image] batch shape: torch.Size([1, 13, 1, 224, 224])
[Image] first_volume: torch.Size([13, 1, 224, 224])
[Image] first_slice: torch.Size([1, 224, 224])
---------------------------------------------
[Image] batch shape: torch.Size([1, 13, 1, 224, 224])
[Image] first_volume: torch.Size([13, 1, 224, 224])
[Image] first_slice: torch.Size([1, 224, 224])


In [8]:
import volumentations

In [13]:
Compose([
    Resize((13,224,224), always_apply=True),
], p=1)

NameError: name 'Resize' is not defined

In [9]:
resize = monai.transforms.Resize((13,1,224,224))

In [33]:
first_volume.shape

torch.Size([13, 1, 224, 224])

In [34]:
resize(first_volume).shape

NotImplementedError: Input Error: Only 3D, 4D and 5D input Tensors supported (got 6D) for the modes: nearest | linear | bilinear | bicubic | trilinear (got area)

In [39]:
def mask_2_onehot(mask, num_classes):
    """
    Transform multiclass mask [1, h, w] to [num_classes, h, w]
    """
    class_slice = []
    for c_class in range(num_classes+1):
        class_slice.append( ((mask == c_class) * 1) )
    return torch.cat(class_slice)

In [40]:
first_slice_mask_onehot = cat_mask_2_onehot(first_slice_mask, 3)

In [50]:
first_slice_mask_onehot.permute(1,2,0).view(-1, 4).shape

torch.Size([50176, 4])

In [53]:
compute_covariance( first_slice_mask_onehot.permute(1,2,0).view(-1, 4).float() )+1

tensor([[1.9063, 1.0000, 1.0000, 1.0000],
        [1.0000, 1.0483, 1.0000, 1.0000],
        [1.0000, 1.0000, 1.0160, 1.0000],
        [1.0000, 1.0000, 1.0000, 1.0294]])

In [49]:
mask_list_A, img_list_A = get_data("A", normalization="none")
mask_list_B, img_list_B = get_data("B", normalization="none")
#img_list_C = get_data("C", normalization="none", data_mod="_unlabeled")

In [50]:
mask_list_B.shape

torch.Size([100, 1, 224, 224])

In [37]:
torch.randn((2,10))

tensor([[ 1.1224, -0.1344, -1.1159,  0.5518, -0.1867,  0.5311, -0.3326,  0.3444,
         -2.7127,  0.3377]])

In [54]:
compute_covariance(torch.zeros((10,2)))

tensor([[0., 0.],
        [0., 0.]])